<a href="https://colab.research.google.com/github/Floating-City/Final_Project_For_MATH-3790/blob/main/Word_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
import nltk
from nltk.util import ngrams
from collections import defaultdict, Counter
from datasets import load_dataset

class MarkovModelTagger:
    def __init__(self):
        self.model = defaultdict(Counter)

    def train(self, tagged_sentences):
        for sentence in tagged_sentences:
            # Skip sentences with fewer than 3 tokens
            if len(sentence) < 3:
                continue
            for ngram in ngrams(sentence, 3, pad_left=True, pad_right=True):
                if None in ngram:
                    continue
                (w1, t1), (w2, t2), (w3, t3) = ngram
                self.model[(t1, t2)][t3] += 1

    def tag(self, sentence):
        tagged_sentence = []
        prev_tags = (None, None)
        for word in sentence:
            if prev_tags in self.model and self.model[prev_tags]:
                tag = self.model[prev_tags].most_common(1)[0][0]
            elif (prev_tags[1],) in self.model and self.model[(prev_tags[1],)]:
                tag = self.model[(prev_tags[1],)].most_common(1)[0][0]
            elif word in self.model and self.model[word]:
                tag = self.model[word].most_common(1)[0][0]
            else:
                tag = nltk.pos_tag([word])[0][1]  # Use NLTK's pos_tag as a fallback
            tagged_sentence.append((word, tag))
            prev_tags = (prev_tags[1], tag)
        return tagged_sentence

    def tag_sentence(self, sentence):
        words = nltk.word_tokenize(sentence)
        return self.tag(words)

# Load the dataset
dataset = load_dataset('universal_dependencies', 'en_ewt', split='train')

# Extract sentences
sentences = [example['text'] for example in dataset]

# Tokenize and tag sentences
tagged_sentences = [nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in sentences]

# Initialize and train the tagger
tagger = MarkovModelTagger()
tagger.train(tagged_sentences)

# Tag a sentence
sentence = "This is a sample test sentence"
print(tagger.tag_sentence(sentence))

[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sample', 'NN'), ('test', 'IN'), ('sentence', 'DT')]


In [7]:
import nltk
from nltk.util import ngrams
from collections import defaultdict, Counter
from datasets import load_dataset

class MarkovModelTagger:
    def __init__(self):
        self.model = defaultdict(Counter)

    def train(self, tagged_sentences):
        for sentence in tagged_sentences:
            # Skip sentences with fewer than 3 tokens
            if len(sentence) < 3:
                continue
            for ngram in ngrams(sentence, 3, pad_left=True, pad_right=True):
                if None in ngram:
                    continue
                (w1, t1), (w2, t2), (w3, t3) = ngram
                self.model[(t1, t2)][t3] += 1

    def tag(self, sentence):
        tagged_sentence = []
        prev_tags = (None, None)
        for word in sentence:
            if prev_tags in self.model and self.model[prev_tags]:
                tag = self.model[prev_tags].most_common(1)[0][0]
            elif (prev_tags[1],) in self.model and self.model[(prev_tags[1],)]:
                tag = self.model[(prev_tags[1],)].most_common(1)[0][0]
            elif word in self.model and self.model[word]:
                tag = self.model[word].most_common(1)[0][0]
            else:
                tag = nltk.pos_tag([word])[0][1]  # Use NLTK's pos_tag as a fallback
            tagged_sentence.append((word, tag))
            prev_tags = (prev_tags[1], tag)
        return tagged_sentence

    def tag_sentence(self, sentence):
        words = nltk.word_tokenize(sentence)
        return self.tag(words)
# Load the dataset
dataset = load_dataset('universal_dependencies', 'en_ewt', split='train')

sentences = [example['text'] for example in dataset]
tagged_sentences = [nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in sentences]

# Initialize and train the tagger
tagger = MarkovModelTagger()
tagger.train(tagged_sentences)

from nltk import pos_tag, word_tokenize
from nltk.tag import ClassifierBasedPOSTagger
from sklearn.model_selection import train_test_split
from nltk.metrics.scores import precision, recall, f_measure
from scipy.stats import zscore
import numpy as np

dataset = load_dataset('universal_dependencies', 'en_ewt', split='train')

sentences = [example['text'] for example in dataset]
tagged_sentences = [nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in sentences]

train_set, test_set = train_test_split(tagged_sentences, test_size=0.2, random_state=42)

# Create a new ClassifierBasedPOSTagger
tagger = ClassifierBasedPOSTagger(train=train_set)

# Test the tagger
accuracy = tagger.evaluate(test_set)
print(f"ClassifierBasedPOSTagger accuracy: {accuracy:.2f}")

# Compare with MarkovModelTagger
tagger_markov = MarkovModelTagger()
tagger_markov.train(train_set)
accuracy_markov = sum(t1 == t2 for sent in test_set for (w1, t1), (w2, t2) in zip(sent, tagger_markov.tag([w for w, t in sent]))) / sum(len(sent) for sent in test_set)
print(f"MarkovModelTagger accuracy: {accuracy_markov:.2f}")

# Perform a z-test
z_score = (accuracy - accuracy_markov) / np.sqrt((accuracy * (1 - accuracy) / len(test_set)) + (accuracy_markov * (1 - accuracy_markov) / len(test_set)))
print(f"z-score: {z_score:.2f}")


<ipython-input-7-5dd937de4052>:70: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = tagger.evaluate(test_set)


ClassifierBasedPOSTagger accuracy: 0.92
MarkovModelTagger accuracy: 0.20
z-score: 73.33
